# Constants


In [70]:

import pandas as pd
import os
import shutil
from os import path as osp

import torch
from tiatoolbox.utils.misc import select_device
import random
import numpy as np
from pathlib import Path
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import joblib
import json
import glob

# from src.intensity import add_features_and_create_new_dicts

from src.featureextraction import get_cell_features, add_features_and_create_new_dicts
from src.train import stratified_split, recur_find_ext, run_once, rm_n_mkdir ,reset_logging
from src.graph_construct import create_graph_with_pooled_patch_nodes, get_pids_labels_for_key


ON_GPU = False
device = select_device(on_gpu=ON_GPU)

SEED = 5
random.seed(SEED)
rng = np.random.default_rng(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)


BASEDIR = '/home/amrit/data/proj_data/MLG_project/DLBCL-Morph'


STAIN = 'MYC'
# STAIN = 'BCL2'


FIDIR = f'{BASEDIR}/outputs'
CLINPATH = f'{BASEDIR}/clinical_data_cleaned.csv'
ANNPATH = f'{BASEDIR}/annotations_clean.csv'
FEATSDIR = f'{BASEDIR}/outputs/files/{STAIN}'
FEATSCALERPATH = f"{FEATSDIR}/0_feat_scaler.npz"
PATCH_SIZE = 224
OUTPUT_SIZE = PATCH_SIZE*8

WORKSPACE_DIR = Path(BASEDIR)
# GRAPH_DIR = WORKSPACE_DIR / f"graphs{STAIN}" 
# LABELS_PATH = WORKSPACE_DIR / "graphs/0_labels.txt"


# Graph construction
# PATCH_SIZE = 300
SKEW_NOISE = 0.0001
MIN_CELLS_PER_PATCH = 10
CONNECTIVITY_DISTANCE = 500


GRAPHSDIR = Path(f'{BASEDIR}/graphs/{STAIN}')
LABELSPATH = f'{BASEDIR}/graphs/{STAIN}_labels.json'

NUM_EPOCHS = 100
NUM_NODE_FEATURES = 128
NCLASSES = 3

TRAIN_DIR = WORKSPACE_DIR / "training"
SPLIT_PATH = TRAIN_DIR / f"splits_{STAIN}.dat"
RUN_OUTPUT_DIR = TRAIN_DIR / f"session_{STAIN}_{datetime.now().strftime('%m_%d_%H_%M')}"
RUN_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR = RUN_OUTPUT_DIR / "model"

# Extract features: 20 + 4 minutes

In [71]:
# read annotation csv, filter, and process intensity features
df = pd.read_csv(ANNPATH)

df = df[df['stain'] == STAIN]
df['area'] = (df['xe'] - df['xs']) *  (df['ye'] - df['ys'])/10000
df = df[df['area'] >= 150]  
df = df[df['xs']  >=0 ]
df = df[df['ys']  >=0 ]
df = df[df['xe']  >=0 ]
df = df[df['ye']  >=0 ]

df = df.reset_index()
##########

###############
# add intensity features
start_index = 0
end_index = len(df.index)

datpaths = []
imgpaths = []
updatpaths = []

# for index in range(start_index, 1):
for index in range(start_index, end_index):
    df_index = df['index'][index]
    patient_id = df['patient_id'][index]
    stain = df['stain'][index]
    tma_id = df['tma_id'][index]
    unique_id = str(patient_id) + '_' + stain + '_' + str(df_index)

    img_file_name = f"{FIDIR}/images/{patient_id}/{patient_id}_{stain}_{tma_id}_{OUTPUT_SIZE}_{df_index}.png"
    dat_file_name = f"{FIDIR}/files/{stain}/{patient_id}/{df_index}/0.dat"
    updat_file_name = f"{FIDIR}/files/{stain}/{patient_id}/{df_index}/{unique_id}.dat"

    datpaths.append(dat_file_name)
    imgpaths.append(img_file_name)
    updatpaths.append(updat_file_name)
    


In [72]:
updatpaths[:3]

['/home/amrit/data/proj_data/MLG_project/DLBCL-Morph/outputs/files/MYC/13902/513/13902_MYC_513.dat',
 '/home/amrit/data/proj_data/MLG_project/DLBCL-Morph/outputs/files/MYC/13902/514/13902_MYC_514.dat',
 '/home/amrit/data/proj_data/MLG_project/DLBCL-Morph/outputs/files/MYC/13924/515/13924_MYC_515.dat']

In [73]:
# 16 minutes
# shutil.rmtree(FEATSDIR)
# if not osp.exists(FEATSDIR):
#     os.makedirs(FEATSDIR)
add_features_and_create_new_dicts(datpaths, imgpaths, updatpaths)

 91%|█████████▏| 333/364 [19:32<02:00,  3.89s/it]

updatpath: /home/amrit/data/proj_data/MLG_project/DLBCL-Morph/outputs/files/MYC/13981/2615/13981_MYC_2615.dat
datpath: /home/amrit/data/proj_data/MLG_project/DLBCL-Morph/outputs/files/MYC/13981/2615/0.dat
imgpath: /home/amrit/data/proj_data/MLG_project/DLBCL-Morph/outputs/images/13981/13981_MYC_TA234_1792_2615.png
valid_cells_frac 0.9998890614599512 = 1-1/9014


100%|██████████| 364/364 [21:24<00:00,  3.53s/it]


In [75]:
# 4 minutes
gns = StandardScaler()
for featpath in tqdm(updatpaths):
    try:
        celldatadict = joblib.load(featpath)
        cellsfeats = np.array([v['intensity_feats'] for k, v in celldatadict.items()])
        gns.partial_fit(cellsfeats)
    except:
        print(featpath)

np.savez(FEATSCALERPATH, mean=gns.mean_, var=gns.var_)

# dd = np.load(FEATSCALERPATH)
# print(dd['mean'], gns.mean_)
# print(dd['var'], gns.var_)

100%|██████████| 364/364 [03:32<00:00,  1.71it/s]


In [76]:
FEATSCALERPATH

'/home/amrit/data/proj_data/MLG_project/DLBCL-Morph/outputs/files/MYC/0_feat_scaler.npz'

# Prepare graphs : 4 minutes

In [66]:
FEATSDIR

'/home/amrit/data/proj_data/MLG_project/DLBCL-Morph/outputs/files/MYC'

In [77]:

# Graph construction
PATCH_SIZE = 300
SKEW_NOISE = 0.0001
MIN_CELLS_PER_PATCH = 10
CONNECTIVITY_DISTANCE = 500


# divide into quantiles
df = pd.read_csv(CLINPATH)
df_labels = get_pids_labels_for_key(df, key='OS', nclasses=3)

# # save paths
# featpaths = np.sort(glob.glob(f'{FEATSDIR}/**/*.dat', recursive=True)) #np.sort(glob.glob(f'{FEATSDIR}/*.dat'))
# featpaths = [x if "/0.dat" not in x for x in featpaths]
featpaths = np.sort(glob.glob(f'{FEATSDIR}/**/*.dat', recursive=True)) #np.sort(glob.glob(f'{FEATSDIR}/*.dat'))
featpaths = [x for x in featpaths if ("/0.dat" not in x) and ("/file_map.dat" not in x)]
featpaths
pids = [int(osp.basename(featpath).split('_')[0]) for featpath in featpaths]
df_featpaths = pd.DataFrame(zip(pids, featpaths), columns=['patient_id', 'featpath'])

# merge to find datapoints with graph data and labels
df_data = df_featpaths.merge(df_labels, on='patient_id')
# df_data = df_data[:12]

featpaths_data = df_data['featpath'].to_list()
labels_data = df_data['OS_class'].to_list()
display(df_featpaths)

outgraphpaths_data = [f"{GRAPHSDIR}/{osp.basename(featpath).split('.')[0]}.json" for featpath in featpaths_data]

# save labels
labels_dict = {osp.basename(graphpath): label for graphpath, label in zip(outgraphpaths_data, labels_data)}
with open(LABELSPATH, 'w') as f:
    json.dump(labels_dict, f)


# read normalizer stats from file and pass to fn
dd = np.load(FEATSCALERPATH)
cell_feat_norm_stats = (dd['mean'], dd['var'])

# create final graphs data
shutil.rmtree(GRAPHSDIR)
if not osp.exists(GRAPHSDIR):
    os.makedirs(GRAPHSDIR)
    create_graph_with_pooled_patch_nodes(
        featpaths_data,
        labels_data,
        outgraphpaths_data,
        PATCH_SIZE,
        cell_feat_norm_stats=cell_feat_norm_stats,
        MIN_CELLS_PER_PATCH= MIN_CELLS_PER_PATCH,
        CONNECTIVITY_DISTANCE = CONNECTIVITY_DISTANCE
    )
    # wont work in parallel mode
    # print(np.mean(global_patch_stats), np.std(global_patch_stats))

,patient_id,featpath
0,13901,/home/amrit/data/proj_data/MLG_project/DLBCL-M...
1,13901,/home/amrit/data/proj_data/MLG_project/DLBCL-M...
2,13902,/home/amrit/data/proj_data/MLG_project/DLBCL-M...
3,13902,/home/amrit/data/proj_data/MLG_project/DLBCL-M...
4,13903,/home/amrit/data/proj_data/MLG_project/DLBCL-M...
...,...,...
359,26857,/home/amrit/data/proj_data/MLG_project/DLBCL-M...
360,26858,/home/amrit/data/proj_data/MLG_project/DLBCL-M...
361,26858,/home/amrit/data/proj_data/MLG_project/DLBCL-M...
362,26859,/home/amrit/data/proj_data/MLG_project/DLBCL-M...


  0%|          | 0/319 [00:00<?, ?it/s]|2023-11-21|22:13:33.434| [WARNING] /home/amrit/anaconda3/envs/DL2/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()

|2023-11-21|22:13:33.435| [WARNING] /home/amrit/anaconda3/envs/DL2/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-usin

In [79]:
len(labels_data)

319

# Create data splits

In [28]:
GRAPHSDIR

'/home/amrit/data/proj_data/MLG_project/DLBCL-Morph/graphs/BCL2'

In [80]:
wsi_paths = recur_find_ext(GRAPHSDIR, [".json"])
wsi_names = [Path(v).stem for v in wsi_paths]
assert len(wsi_paths) > 0, "No files found."  # noqa: S101

len(wsi_paths) , len(wsi_names) , wsi_names[:5]

(319,
 319,
 ['13901_MYC_532',
  '13901_MYC_557',
  '13902_MYC_513',
  '13902_MYC_514',
  '13903_MYC_535'])

In [81]:
NUM_FOLDS = 1
TEST_RATIO = 0.2
TRAIN_RATIO = 0.8 * 0.7
VALID_RATIO = 0.8 * 0.3

# if SPLIT_PATH and os.path.exists(SPLIT_PATH):
#     splits = joblib.load(SPLIT_PATH)
# else:
x = np.array(wsi_names)
with open(LABELSPATH, 'r') as f:
    labels_dict = json.load(f)
print(labels_dict)
y = np.array([labels_dict[wsi_name+'.json'] for wsi_name in wsi_names])
y[np.where(y==-1)] = 0
splits = stratified_split(x, y, TRAIN_RATIO, VALID_RATIO, TEST_RATIO, NUM_FOLDS)
joblib.dump(splits, SPLIT_PATH)


{'13901_MYC_532.json': 0, '13901_MYC_557.json': 0, '13902_MYC_513.json': 2, '13902_MYC_514.json': 2, '13903_MYC_535.json': 2, '13903_MYC_560.json': 2, '13904_MYC_539.json': 1, '13904_MYC_550.json': 1, '13908_MYC_527.json': 1, '13908_MYC_555.json': 1, '13911_MYC_520.json': 2, '13911_MYC_533.json': 2, '13912_MYC_530.json': 0, '13913_MYC_536.json': 0, '13913_MYC_554.json': 0, '13914_MYC_534.json': 0, '13914_MYC_556.json': 0, '13915_MYC_516.json': 2, '13915_MYC_547.json': 2, '13917_MYC_526.json': 2, '13917_MYC_551.json': 2, '13919_MYC_518.json': 0, '13919_MYC_528.json': 0, '13920_MYC_549.json': 2, '13922_MYC_558.json': 2, '13923_MYC_525.json': 2, '13923_MYC_538.json': 2, '13924_MYC_515.json': 1, '13924_MYC_548.json': 1, '13952_MYC_2587.json': 1, '13952_MYC_2589.json': 1, '13953_MYC_2611.json': 2, '13953_MYC_2639.json': 2, '13954_MYC_2592.json': 2, '13955_MYC_2637.json': 1, '13957_MYC_2646.json': 1, '13958_MYC_2600.json': 2, '13958_MYC_2640.json': 2, '13959_MYC_2593.json': 1, '13959_MYC_264

['/home/amrit/data/proj_data/MLG_project/DLBCL-Morph/training/splits_MYC.dat']

In [82]:
y

array([0, 0, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0,
       0, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2,
       1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 0, 0, 1, 1, 0, 1,
       1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 2,
       2, 1, 1, 2, 2, 1, 1, 0, 0, 2, 2, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1,

In [83]:
from src.utils import load_json, rm_n_mkdir, mkdir, recur_find_ext
from src.dset import SlideGraphDataset, stratified_split, StratifiedSampler
import warnings
warnings.filterwarnings("ignore")

nodes_preproc_func = None

def sample_data(  # noqa: C901, PLR0912, PLR0915
    dataset_dict: dict,
    GRAPH_DIR = None) -> list:
    
    for subset_name, subset in dataset_dict.items():
        ds = SlideGraphDataset(subset, mode="train", preproc=nodes_preproc_func, graph_dir=GRAPH_DIR)
    
    return ds

splits = joblib.load(SPLIT_PATH)

for split_idx, split in enumerate(splits):
    new_split = {
                "train": split["train"],
                "infer-train": split["train"],
                "infer-valid-A": split["valid"],
                "infer-valid-B": split["test"],
            }
    ds = sample_data(new_split, GRAPHSDIR)

sample = ds.__getitem__(3)

In [84]:
display(sample['graph'])
display(sample['graph']['x'])
display(sample['graph']['edge_index'])
display(sample['graph']['y'])
display(sample['graph']['coordinates'])


Data(x=[73, 128], edge_index=[2, 382], y=1, coordinates=[73, 2])

tensor([[-0.3422,  0.7238, -0.0408,  ...,  1.4282,  1.5008,  2.8769],
        [-0.6838,  0.5138, -0.5111,  ...,  0.8528,  0.8395,  2.9245],
        [-0.4224,  0.4728, -0.3540,  ...,  1.3297,  1.3726,  3.8084],
        ...,
        [-0.2560,  0.6157, -0.5808,  ...,  1.5298,  1.6022,  3.3516],
        [-0.4646,  0.7058, -0.5871,  ...,  1.3005,  1.3412,  2.8971],
        [-0.4629,  0.6778, -0.4312,  ...,  1.5494,  1.5737,  3.9638]],
       dtype=torch.float64)

tensor([[ 0,  0,  0,  0,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  4,  4,  4,
          4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,
          7,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11,
         11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14,
         14, 14, 14, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 17,
         18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 21,
         21, 21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 24, 24, 24,
         24, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 27, 27, 27, 28, 28,
         28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 31,
         31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 33,
         34, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36, 36, 36, 37, 37, 37, 37, 37,
         37, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 40, 40, 40, 40,
         40, 40, 41, 41, 41,

tensor(1)

tensor([[ 258.1287, 1120.5666],
        [ 214.6071, 1364.1682],
        [ 202.5461, 1628.5665],
        [ 244.8567, 1953.5732],
        [ 501.5717,  756.0352],
        [ 472.8767, 1063.9933],
        [ 448.5676, 1355.8731],
        [ 444.8090, 1653.3079],
        [ 463.7815, 1955.0860],
        [ 474.5978, 2236.8770],
        [ 758.0824,  524.7156],
        [ 743.2552,  771.1221],
        [ 731.7738, 1048.6695],
        [ 753.5197, 1359.7750],
        [ 757.3856, 1656.0150],
        [ 766.4515, 1962.2483],
        [ 764.1809, 2245.5292],
        [ 774.0712, 2531.1890],
        [1053.9456,  464.3224],
        [1042.0817,  739.2097],
        [1058.4759, 1073.2831],
        [1044.9298, 1360.5932],
        [1060.9097, 1647.9690],
        [1042.7322, 1945.6238],
        [1051.6896, 2244.1999],
        [1058.7076, 2556.3594],
        [1089.5511, 2760.4148],
        [1342.5973,  452.9947],
        [1357.7719,  740.3242],
        [1367.3723, 1059.0892],
        [1347.7761, 1353.0263],
        

# Train model

In [86]:
# # we must define the function after training/loading
# def nodes_preproc_func(node_features: np.ndarray) -> np.ndarray:
#     """Pre-processing function for nodes."""
#     return node_scaler.transform(node_features)
nodes_preproc_func = None


splits = joblib.load(SPLIT_PATH)
loader_kwargs = {
    "num_workers": 6,
    "batch_size": 32,
}
# arch_kwargs = {
#     "dim_features": NUM_NODE_FEATURES,
#     "dim_target": NCLASSES,
#     "layers": [32, 32, 16, 8],
#     "dropout": 0.3,
#     "pooling": "mean",
#     "conv": "EdgeConv",
#     "aggr": "max",
# }

arch_kwargs = {
    "dim_features": NUM_NODE_FEATURES,
    "dim_target": NCLASSES,
    "layers": [64, 32, 32],
    "dropout": 0.3,
    "pooling": "mean",
    "conv": "EdgeConv",
    "aggr": "max",
}
optim_kwargs = {
    "lr": 5.0e-4,
    "weight_decay": 1.0e-4,
}

NUM_EPOCHS = 100
# if not MODEL_DIR.exists() or True:
for split_idx, split in enumerate(splits):
        new_split = {
            "train": split["train"],
            "infer-train": split["train"],
            "infer-valid-A": split["valid"],
            "infer-valid-B": split["test"],
        }
        MODEL_DIR = Path(MODEL_DIR) 
        split_save_dir = MODEL_DIR / f"{split_idx:02d}/"
        rm_n_mkdir(split_save_dir)
        reset_logging(split_save_dir)
        run_once(
            new_split,
            NUM_EPOCHS,
            save_dir=split_save_dir,
            arch_kwargs=arch_kwargs,
            loader_kwargs=loader_kwargs,
            optim_kwargs=optim_kwargs,
            on_gpu=ON_GPU,
            GRAPH_DIR=GRAPHSDIR
        )

|2023-11-21|22:18:44.574| [INFO] EPOCH: 000


SlideGraphArch(
  (first_h): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (nns): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=128, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (convs): ModuleList(
    (0): EdgeConv(nn=Sequential(
      (0): Linear(in_features=128, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    ))
    (1): EdgeConv(nn=Sequential(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05,

100%|██████████| 5/5 [00:00<00:00, 13.16it/s]
|2023-11-21|22:18:44.957| [INFO] train-EMA-loss: 1.412522
|2023-11-21|22:18:45.263| [INFO] infer-train-accuracy: 0.292135
|2023-11-21|22:18:45.513| [INFO] infer-valid-A-accuracy: 0.259740
|2023-11-21|22:18:45.733| [INFO] infer-valid-B-accuracy: 0.265625
|2023-11-21|22:18:45.961| [INFO] EPOCH: 001


best_score {'infer-train-accuracy': 0, 'infer-valid-A-accuracy': 0, 'infer-valid-B-accuracy': 0}


100%|██████████| 5/5 [00:00<00:00, 13.25it/s]
|2023-11-21|22:18:46.341| [INFO] train-EMA-loss: 1.287859
|2023-11-21|22:18:46.648| [INFO] infer-train-accuracy: 0.331461
|2023-11-21|22:18:46.873| [INFO] infer-valid-A-accuracy: 0.324675
|2023-11-21|22:18:47.125| [INFO] infer-valid-B-accuracy: 0.359375
|2023-11-21|22:18:47.343| [INFO] EPOCH: 002


...

best_score {'infer-train-accuracy': 0.9831460674157303, 'infer-valid-A-accuracy': 0.5454545454545454, 'infer-valid-B-accuracy': 0.6875}


100%|██████████| 5/5 [00:00<00:00, 13.20it/s]
|2023-11-21|22:21:05.211| [INFO] train-EMA-loss: 0.203776
|2023-11-21|22:21:05.548| [INFO] infer-train-accuracy: 0.943820
|2023-11-21|22:21:05.822| [INFO] infer-valid-A-accuracy: 0.506494
|2023-11-21|22:21:06.101| [INFO] infer-valid-B-accuracy: 0.562500
|2023-11-21|22:21:06.371| [INFO] EPOCH: 099


best_score {'infer-train-accuracy': 0.9831460674157303, 'infer-valid-A-accuracy': 0.5454545454545454, 'infer-valid-B-accuracy': 0.6875}


100%|██████████| 5/5 [00:00<00:00, 12.34it/s]
|2023-11-21|22:21:06.779| [INFO] train-EMA-loss: 0.169980
|2023-11-21|22:21:07.076| [INFO] infer-train-accuracy: 0.910112
|2023-11-21|22:21:07.352| [INFO] infer-valid-A-accuracy: 0.506494
|2023-11-21|22:21:07.614| [INFO] infer-valid-B-accuracy: 0.546875


best_score {'infer-train-accuracy': 0.9831460674157303, 'infer-valid-A-accuracy': 0.5454545454545454, 'infer-valid-B-accuracy': 0.6875}
